In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lightweight_charts import Chart
from sklearn.linear_model import LinearRegression
from sklearn.cluster import DBSCAN

In [2]:
df=pd.read_csv("data/btc_futures_data.csv")
df['Date'] = pd.to_datetime(df['Open Time']) # 꺼낸 데이터 프레임의 날짜 변수 형변환을 진행합니다.
df.set_index(['Date'], inplace = True)
df=df[['Open', 'High', 'Low', 'Close', 'Volume']]
df

,Open,High,Low,Close,Volume
Date,,,,,
2019-09-09 01:00:00,10000.00,10357.53,10000.00,10340.12,471.661
2019-09-09 05:00:00,10340.12,10412.65,10324.77,10391.63,2624.630
2019-09-09 09:00:00,10316.62,10316.68,10267.37,10297.89,779.449
2019-09-09 13:00:00,10297.89,10316.34,10092.71,10149.47,2664.789
2019-09-09 17:00:00,10149.47,10475.54,10077.22,10414.60,2466.485
...,...,...,...,...,...
2024-11-12 21:00:00,87229.50,87933.30,85159.90,86659.90,131106.429
2024-11-13 01:00:00,86660.00,89830.00,85792.50,88762.80,98262.570
2024-11-13 05:00:00,88762.90,89920.00,87384.60,87972.00,62526.904


In [3]:
df['EMA9'] = df['Close'].ewm(span=9, adjust=False).mean()
df['EMA60'] = df['Close'].ewm(span=60, adjust=False).mean()
df['EMA200'] = df['Close'].ewm(span=200, adjust=False).mean()

In [4]:

df['UD'] = [df['Close'].iloc[i] - df['Close'].iloc[i-1] if i > 0 else 0 for i in range(len(df['Close']))]

df['RSI_U'] = df['UD'].apply(lambda x : x if x > 0 else 0)
df['RSI_D'] = df['UD'].apply(lambda x : x * (-1) if x < 0 else 0)

df['RSI_AU'] = df['RSI_U'].rolling(14).mean() # 평균을 계산하기 위한 기간을 14일로 설정하였습니다.
df['RSI_AD'] = df['RSI_D'].rolling(14).mean()

df['RSI'] = df.apply(lambda x : x['RSI_AU'] / (x['RSI_AU'] + x['RSI_AD']) * 100, 1)
df['RSI_SMA'] = df['RSI'].rolling(window=9).mean()


In [5]:
df['StochRSI_%K'] = (df['RSI']-df['RSI'].rolling(window=14).min())/(df['RSI'].rolling(window=14).max()-df['RSI'].rolling(window=14).min())
df['StochRSI_%D'] = df['StochRSI_%K'].rolling(window=3).mean()

In [6]:

df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()

df['MACD'] = df['EMA12'] - df['EMA26']
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
df['MACD_Oscillator'] = df['MACD'] - df['Signal']

In [7]:
df=df.drop(['UD', 'RSI_U', 'RSI_D', 'RSI_AU', 'RSI_AD', 'EMA12', 'EMA26'], axis=1)

In [8]:
df

,Open,High,Low,Close,Volume,EMA9,EMA60,EMA200,RSI,RSI_SMA,StochRSI_%K,StochRSI_%D,MACD,Signal,MACD_Oscillator
Date,,,,,,,,,,,,,,,
2019-09-09 01:00:00,10000.00,10357.53,10000.00,10340.12,471.661,10340.120000,10340.120000,10340.120000,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000
2019-09-09 05:00:00,10340.12,10412.65,10324.77,10391.63,2624.630,10350.422000,10341.808852,10340.632537,NaN,NaN,NaN,NaN,4.109060,0.821812,3.287248
2019-09-09 09:00:00,10316.62,10316.68,10267.37,10297.89,779.449,10339.915600,10340.368890,10340.207238,NaN,NaN,NaN,NaN,-0.196253,0.618199,-0.814452
2019-09-09 13:00:00,10297.89,10316.34,10092.71,10149.47,2664.789,10301.826480,10334.109910,10338.309355,NaN,NaN,NaN,NaN,-15.406897,-2.586820,-12.820077
2019-09-09 17:00:00,10149.47,10475.54,10077.22,10414.60,2466.485,10324.381184,10336.748929,10339.068466,NaN,NaN,NaN,NaN,-5.998519,-3.269160,-2.729359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-12 21:00:00,87229.50,87933.30,85159.90,86659.90,131106.429,85864.807965,77334.121790,71001.377416,75.575152,87.093602,0.000000,0.350259,3292.476825,2832.628539,459.848285
2024-11-13 01:00:00,86660.00,89830.00,85792.50,88762.80,98262.570,86444.406372,77708.832551,71178.107989,77.126402,86.425592,0.083332,0.070331,3352.754050,2936.653641,416.100409
2024-11-13 05:00:00,88762.90,89920.00,87384.60,87972.00,62526.904,86749.925098,78045.329845,71345.211392,74.389587,85.033878,0.000000,0.027777,3298.688012,3009.060516,289.627497


In [9]:
df.fillna(0)

,Open,High,Low,Close,Volume,EMA9,EMA60,EMA200,RSI,RSI_SMA,StochRSI_%K,StochRSI_%D,MACD,Signal,MACD_Oscillator
Date,,,,,,,,,,,,,,,
2019-09-09 01:00:00,10000.00,10357.53,10000.00,10340.12,471.661,10340.120000,10340.120000,10340.120000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2019-09-09 05:00:00,10340.12,10412.65,10324.77,10391.63,2624.630,10350.422000,10341.808852,10340.632537,0.000000,0.000000,0.000000,0.000000,4.109060,0.821812,3.287248
2019-09-09 09:00:00,10316.62,10316.68,10267.37,10297.89,779.449,10339.915600,10340.368890,10340.207238,0.000000,0.000000,0.000000,0.000000,-0.196253,0.618199,-0.814452
2019-09-09 13:00:00,10297.89,10316.34,10092.71,10149.47,2664.789,10301.826480,10334.109910,10338.309355,0.000000,0.000000,0.000000,0.000000,-15.406897,-2.586820,-12.820077
2019-09-09 17:00:00,10149.47,10475.54,10077.22,10414.60,2466.485,10324.381184,10336.748929,10339.068466,0.000000,0.000000,0.000000,0.000000,-5.998519,-3.269160,-2.729359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-12 21:00:00,87229.50,87933.30,85159.90,86659.90,131106.429,85864.807965,77334.121790,71001.377416,75.575152,87.093602,0.000000,0.350259,3292.476825,2832.628539,459.848285
2024-11-13 01:00:00,86660.00,89830.00,85792.50,88762.80,98262.570,86444.406372,77708.832551,71178.107989,77.126402,86.425592,0.083332,0.070331,3352.754050,2936.653641,416.100409
2024-11-13 05:00:00,88762.90,89920.00,87384.60,87972.00,62526.904,86749.925098,78045.329845,71345.211392,74.389587,85.033878,0.000000,0.027777,3298.688012,3009.060516,289.627497


In [11]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots